In [1]:
import os, sys
sys.path.append( os.path.join('..','..','src','mlspec_flow') )

from utils.paths import saved_model_dir

In [15]:
flow_dir   = "TaxiTT2017PosTimeCompanyV01"
model_name = "20210521"
numeric_columns     = ['pickup_latitude',
                       'pickup_longitude',
                       'dropoff_latitude',
                       'dropoff_longitude']
categorical_columns = ['trip_start', # [technical] this is in categorical because its TF placeholder for serving must be of type string
                       'company']
label="trip_total"
os.environ["MODEL_PATH"] = saved_model_dir(flow_dir,model_name)

In [36]:
os.environ["MODEL_PATH"]

'gs://mlteam-ml-specialization-2021-taxi/tft_flows/TaxiTT2017PosTimeCompanyV01/models/20210521/1/export/1621855172/'

In [10]:
import json

json_path = "req.json"
instance=dict(list(zip(numeric_columns,[0 for _ in numeric_columns]))+list(zip(categorical_columns, ["" for _ in categorical_columns])))
if "trip_start" in instance:
    instance["trip_start"] = "2017-03-03 Fry-09:00"
# %Y-%m-%d %a-%H:%M
json.dump(
    {"instances":[instance]},
    open(json_path,"w")
)
os.environ["JSON_PATH"]  = json_path

In [11]:
instance

{'pickup_latitude': 0,
 'pickup_longitude': 0,
 'dropoff_latitude': 0,
 'dropoff_longitude': 0,
 'trip_start': '2017-03-03 Fry-09:00',
 'company': ''}

In [13]:
!gcloud ai-platform local predict --model-dir="${MODEL_PATH}" --json-request="${JSON_PATH}" --framework=tensorflow --signature-name="predict"

Instructions for updating:
non-resource variables are not supported in the long term
2021-06-17 12:49:21.717319: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-06-17 12:49:21.717671: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-06-17 12:49:21.717701: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-06-17 12:49:21.717728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mirco-nani-dev-spd): /proc/driver/nvidia/version does not exist
2021-06-17 12:49:21.718204: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices n

# Setup

In [23]:
os.environ["AI_PLATFORM_PREDICTION_MODEL"]   = f"taxi{label}"
os.environ["AI_PLATFORM_PREDICTION_REGION"]  = "europe-west1"
os.environ["AI_PLATFORM_PREDICTION_VERSION"] = f"{flow_dir}_{model_name}"

In [18]:
!gcloud ai-platform models create ${AI_PLATFORM_PREDICTION_MODEL} --region=${AI_PLATFORM_PREDICTION_REGION}  --enable-logging

Using endpoint [https://europe-west1-ml.googleapis.com/]
Created ai platform model [projects/mlteam-ml-specialization-2021/models/taxitrip_total].


In [30]:
!gcloud ai-platform versions create ${AI_PLATFORM_PREDICTION_VERSION} \
--region=${AI_PLATFORM_PREDICTION_REGION} \
--model=${AI_PLATFORM_PREDICTION_MODEL} \
--origin=${MODEL_PATH} \
--runtime-version=2.4

Using endpoint [https://europe-west1-ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [37]:
os.environ["AI_PLATFORM_PREDICTION_VERSION"]

'TaxiTT2017PosTimeCompanyV01_20210521'

In [31]:
import json
import os


instance=dict(list(zip(numeric_columns,[0 for _ in numeric_columns]))+list(zip(categorical_columns, ["" for _ in categorical_columns])))
if "trip_start" in instance:
    instance["trip_start"] = "2017-03-03 Fry-09:00"
model_name=os.environ["AI_PLATFORM_PREDICTION_MODEL"]
region=os.environ["AI_PLATFORM_PREDICTION_REGION"]

os.environ["ENDPOINT"]=F"https://{region}-ml.googleapis.com/v1/projects/mlteam-ml-specialization-2021/models/{model_name}:predict"
# https://stackoverflow.com/questions/58134103/tf-how-do-i-setup-the-model-signature-correctly-for-serving-with-docker
os.environ["REQUEST_BODY"]=json.dumps({
    "instances": [instance],
    'signature_name': 'predict'
})

In [38]:
os.environ["REQUEST_BODY"]

'{"instances": [{"pickup_latitude": 0, "pickup_longitude": 0, "dropoff_latitude": 0, "dropoff_longitude": 0, "trip_start": "2017-03-03 Fry-09:00", "company": ""}], "signature_name": "predict"}'

In [32]:
!curl -d "${REQUEST_BODY}" -X POST ${ENDPOINT}?access_token\="$(gcloud auth application-default print-access-token)"

{
    "predictions": [[18.6645203]
    ]
}

# teardown

In [33]:
! gcloud ai-platform versions delete ${AI_PLATFORM_PREDICTION_VERSION} --model=${AI_PLATFORM_PREDICTION_MODEL} --region=${AI_PLATFORM_PREDICTION_REGION} --quiet

Using endpoint [https://europe-west1-ml.googleapis.com/]
Deleting version [TaxiTT2017PosTimeCompanyV01_20210521]......done.             


In [35]:
! gcloud ai-platform models delete ${AI_PLATFORM_PREDICTION_MODEL} --region=${AI_PLATFORM_PREDICTION_REGION} --quiet

Using endpoint [https://europe-west1-ml.googleapis.com/]
Deleting model [taxitrip_total]...done.                                        
